In [44]:
import pandas as pd
from pprint import pprint
from gensim import corpora
from gensim.models import Word2Vec, Phrases
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_multiple_whitespaces, strip_tags, strip_short

In [45]:
# Import data
train_raw = pd.read_csv('data/train.csv')
test_raw = pd.read_csv('data/test.csv')
valid_raw = pd.read_csv('data/val.csv')

In [46]:
test_preprocess = train_raw.copy()

## Preprocessing

In [47]:
def preprocess(text):
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_punctuation, strip_multiple_whitespaces, strip_tags, strip_short]
    return preprocess_string(text, CUSTOM_FILTERS)

In [48]:
test_preprocess['preprocessed'] = test_preprocess['text'].apply(preprocess)
test_preprocess.head(5)

,id,label,text,preprocessed
0,8901,5,Bennett 's naturalistic performance speaks vol...,"[bennett, naturalistic, performance, speaks, v..."
1,2506,5,"Shot in rich , shadowy black-and-white , Devil...","[shot, rich, shadowy, black, and, white, devil..."
2,2381,5,"More than their unique residences , Home Movie...","[more, than, their, unique, residences, home, ..."
3,1262,3,The movie should be credited with remembering ...,"[the, movie, should, credited, with, rememberi..."
4,2542,4,Audiences are advised to sit near the back and...,"[audiences, are, advised, sit, near, the, back..."


In [49]:
test_dictionary = corpora.Dictionary(test_preprocess['preprocessed'])
print(test_dictionary)
# pprint(test_dictionary.token2id)

Dictionary<14993 unique tokens: ['and', 'any', 'anybody', 'bennett', 'body']...>


## Bigram Transformer to Word2Vec Embedding

In [51]:
bigram_transformer = Phrases(test_preprocess['preprocessed'])
vector_model = Word2Vec(bigram_transformer[test_preprocess['preprocessed']], vector_size=256, window=3, min_count=1, workers=4)

In [52]:
print(vector_model.wv)

KeyedVectors<vector_size=256, 15282 keys>


In [53]:
sims = vector_model.wv.most_similar('advised', topn=10)
pprint(sims)

word_vectors = vector_model.wv
word_vectors.save('word_vectors.kv')

[('general', 0.9841145277023315),
 ('strange', 0.9841052293777466),
 ('boring', 0.984086275100708),
 ('seeking', 0.9840654730796814),
 ('ends', 0.9840561151504517),
 ('brings', 0.9840518832206726),
 ('shows', 0.983998715877533),
 ('bit', 0.9839949011802673),
 ('perfect', 0.9839749932289124),
 ('melodrama', 0.9839625358581543)]


### Modelling

In [54]:
# Load Pretrained Genism Word Vector
from gensim.models import KeyedVectors 

word_vectors = KeyedVectors.load('word_vectors.kv', mmap='r')
print(f"Vocabulary Size: {len(word_vectors.index_to_key)}")

Vocabulary Size: 15282


In [57]:
import numpy as np

embedding_dim = 256  # Same as Word2Vec model
vocab_size = len(word_vectors.index_to_key) + 1  # +1 for padding token

# Initialize embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Mapping each word index to its corresponding vector
word_index = {word: i + 1 for i, word in enumerate(word_vectors.index_to_key)}  # Start from 1
for word, i in word_index.items():
    embedding_matrix[i] = word_vectors[word]

print("Embedding Matrix Shape:", embedding_matrix.shape)

Embedding Matrix Shape: (15283, 256)


In [59]:
import tensorflow

vectorizer = tensorflow.keras.layers.TextVectorization(max_tokens=vocab_size, output_sequence_length=256)
vectorizer.adapt(test_preprocess['preprocessed'].apply(lambda x: ' '.join(x)))

# Example
vectorized_text = vectorizer(['advised audiences near back'])
print(vectorized_text)

tf.Tensor(
[[4156  464  883  188    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0  

In [60]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import random

In [14]:
# Model Architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=200,
                    trainable=True))  # Freezing Word2Vec weights

model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

2025-03-05 19:53:29.109734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-05 19:53:29.110432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-05 19:53:29.110852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 300)          4584900   
                                                                 
 bidirectional (Bidirectiona  (None, 200, 256)         439296    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200, 256)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 32)                4

2025-03-05 19:53:29.310813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2025-03-05 19:53:29.328173: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-05 19:53:29.328682: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

In [77]:
# Function to build model with dynamic hyperparameters
def build_model(vocab_size, embedding_dim, embedding_matrix, units_1, units_2, dropout_rate, learning_rate):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=256,
                        trainable=True))  # Freezing Word2Vec weights

    model.add(Bidirectional(LSTM(units_1, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(96, return_sequences=True), backward_layer=LSTM(96, go_backwards=True, recurrent_regularizer='l2', return_sequences=True, )))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(units_2)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='elu'))
    model.add(Dense(5, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [78]:
base_model = build_model(vocab_size=vocab_size, 
                         embedding_dim=embedding_dim, 
                         embedding_matrix=embedding_matrix, 
                         units_1=128, 
                         units_2=64, 
                         dropout_rate=0.3, 
                         learning_rate=0.001)

2025-03-06 01:01:32.398233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:01:32.399051: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:01:32.399502: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [79]:
# Convert labels to binary (positive or negative sentiment)
# Convert labels 1 → 0, 2 → 1, ..., 5 → 4
y_train = test_preprocess['label'] - 1

x_train = tensorflow.convert_to_tensor(np.array(test_preprocess['preprocessed'].apply(lambda x: ' '.join(x))), dtype=tensorflow.string)
x_train = vectorizer(x_train)

history = base_model.fit(
    x_train, 
    y_train, 
    epochs=5, 
    batch_size=128, 
    validation_split=0.2
)



Epoch 1/5


2025-03-06 01:01:35.485621: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:01:35.486392: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:01:35.486841: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

52/52 [==============================] - ETA: 0s - loss: 2.0542 - accuracy: 0.2676

2025-03-06 01:02:47.855621: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:02:47.856315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:02:47.856783: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

52/52 [==============================] - 80s 1s/step - loss: 2.0542 - accuracy: 0.2676 - val_loss: 1.7425 - val_accuracy: 0.3018
Epoch 2/5
52/52 [==============================] - 82s 2s/step - loss: 1.6016 - accuracy: 0.3240 - val_loss: 1.4176 - val_accuracy: 0.3988
Epoch 3/5
52/52 [==============================] - 78s 1s/step - loss: 1.1828 - accuracy: 0.4638 - val_loss: 1.4251 - val_accuracy: 0.3699
Epoch 4/5
52/52 [==============================] - 76s 1s/step - loss: 0.9512 - accuracy: 0.5509 - val_loss: 1.5143 - val_accuracy: 0.3675
Epoch 5/5
52/52 [==============================] - 74s 1s/step - loss: 0.7169 - accuracy: 0.7076 - val_loss: 1.8924 - val_accuracy: 0.3506


In [80]:
# save the model
base_model.save('models/base_model_v2-02.h5')

## Parameter Tuning with Valid Dataset

In [19]:
# Load Model
from tensorflow.keras.models import load_model

In [20]:
valid = valid_raw.copy()

In [21]:
valid['preprocessed'] = valid['text'].apply(preprocess).apply(lambda x: ' '.join(x))
y_valid = valid['label'] - 1

valid.head(5)

,id,label,text,preprocessed
0,6447,5,At least it 's a fairly impressive debut from ...,least fairly impressive debut from the directo...
1,8503,2,Bland but harmless .,bland but harmless
2,2594,5,"Muccino , who directed from his own screenplay...",muccino who directed from his own screenplay c...
3,6482,2,This mistaken-identity picture is so film-cult...,this mistaken identity picture film culture re...
4,5685,2,"Laconic and very stilted in its dialogue , thi...",laconic and very stilted its dialogue this ind...


In [81]:
model = load_model('models/base_model_v2-02.h5')

x_valid = tensorflow.convert_to_tensor(np.array(valid['preprocessed']), dtype=tensorflow.string)
x_valid = vectorizer(x_valid)

2025-03-06 01:09:02.358217: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:09:02.358933: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:09:02.359354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [91]:
# Hyperparameter grid
param_grid = {
    'units_1': [16, 32, 64, 128, 256],
    'units_2': [16, 32, 64, 128],
    'dropout_rate': [0.2, 0.1, 0.25, 0.15],
    'learning_rate': [0.001, 0.005, 0.05],
    'batch_size': [32, 64, 40, 48, 56, 72, 80]
}

# Random Grid Search
def random_grid_search(X_train, y_train, X_val, y_val, vocab_size, embedding_dim, embedding_matrix, n_trials=10):
    best_acc = 0
    best_params = {}
    
    for _ in range(n_trials):
        units_1 = random.choice(param_grid['units_1'])
        units_2 = random.choice(param_grid['units_2'])
        dropout_rate = random.choice(param_grid['dropout_rate'])
        learning_rate = random.choice(param_grid['learning_rate'])
        batch_size = random.choice(param_grid['batch_size'])
        
        print(f"Training with: Units1={units_1}, Units2={units_2}, Dropout={dropout_rate}, LR={learning_rate}, Batch={batch_size}")
        
        model = build_model(vocab_size, embedding_dim, embedding_matrix, units_1, units_2, dropout_rate, learning_rate)
        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=batch_size, verbose=1)
        _, acc = model.evaluate(X_val, y_val)
        
        print(f"Validation Accuracy: {acc:.4f}")
        
        if acc > best_acc:
            best_acc = acc
            best_params = {'units_1': units_1, 'units_2': units_2, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate, 'batch_size': batch_size}
    
    print(f"Best Accuracy: {best_acc:.4f}")
    print("Best Hyperparameters:", best_params)
    return best_params

## Initiate GridSearch


In [92]:
best_params = random_grid_search(x_train, y_train, x_valid, y_valid, vocab_size, embedding_dim, embedding_matrix, n_trials=3)

Training with: Units1=128, Units2=64, Dropout=0.1, LR=0.001, Batch=56


2025-03-06 01:13:34.393438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:13:34.394276: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:13:34.394707: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5


2025-03-06 01:13:34.599828: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2025-03-06 01:13:34.615754: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:13:34.616274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

149/149 [==============================] - ETA: 0s - loss: 1.7855 - accuracy: 0.2711

...

Epoch 1/5


2025-03-06 01:30:29.489119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:30:29.489726: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:30:29.490154: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

149/149 [==============================] - ETA: 0s - loss: 1.8098 - accuracy: 0.2489

2025-03-06 01:32:17.659207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:32:17.659866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:32:17.660278: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

149/149 [==============================] - 115s 749ms/step - loss: 1.8098 - accuracy: 0.2489 - val_loss: 1.6463 - val_accuracy: 0.2576
Epoch 2/5
149/149 [==============================] - 104s 697ms/step - loss: 1.6109 - accuracy: 0.2634 - val_loss: 1.5862 - val_accuracy: 0.2750
Epoch 3/5
149/149 [==============================] - 100s 671ms/step - loss: 1.6234 - accuracy: 0.2583 - val_loss: 1.6308 - val_accuracy: 0.2750
Epoch 4/5
149/149 [==============================] - 87s 585ms/step - loss: 1.6668 - accuracy: 0.2691 - val_loss: 1.6436 - val_accuracy: 0.2762
Epoch 5/5
56/56 [==============================] - 8s 147ms/step - loss: 1.6780 - accuracy: 0.2750
Validation Accuracy: 0.2750
Best Accuracy: 0.3526
Best Hyperparameters: {'units_1': 16, 'units_2': 16, 'dropout_rate': 0.15, 'learning_rate': 0.001, 'batch_size': 80}


In [93]:
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'units_1': 16, 'units_2': 16, 'dropout_rate': 0.15, 'learning_rate': 0.001, 'batch_size': 80}


In [82]:
# Predictions
valid_predictions = model.predict(x_valid)

# Convert predictions to labels
valid_predictions = [np.argmax(pred) + 1 for pred in valid_predictions]

# Calculate accuracy
valid_accuracy = sum(valid_predictions == y_valid) / len(y_valid)

print(f"Validation Accuracy: {valid_accuracy}")

2025-03-06 01:09:09.556178: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:09:09.556941: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:09:09.557402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

56/56 [==============================] - 10s 170ms/step
Validation Accuracy: 0.266029246344207


## Test Model to Submission

In [94]:
best_params_model = build_model(vocab_size, embedding_dim, embedding_matrix, best_params['units_1'], best_params['units_2'], best_params['dropout_rate'], best_params['learning_rate'])
manual_params_model = build_model(vocab_size, embedding_dim, embedding_matrix, 128, 64, 0.3, 0.001)


best_params_model.save('models/best_params_model.h5')
manual_params_model.save('models/manual_params_model.h5')

2025-03-06 01:40:25.633108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:40:25.633874: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:40:25.634306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [83]:
test = test_raw.copy()

In [84]:
test['preprocessed'] = test['text'].apply(preprocess)
test['preprocessed'] = test['preprocessed'].apply(lambda x: ' '.join(x))

In [96]:
# model = load_model('best_params_model.h5')
model = load_model('models/best_params_model.h5')

2025-03-06 01:40:57.802288: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:40:57.803028: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:40:57.803442: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [97]:
# predict the sentiment

result = model.predict(vectorizer(np.array(test['preprocessed'])))

2025-03-06 01:41:06.648348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-06 01:41:06.649134: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-06 01:41:06.649588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

56/56 [==============================] - 7s 109ms/step


In [98]:
import numpy as np

# Convert probabilities into sentiment labels (0 to 4)
predicted_labels = np.argmax(result, axis=1)

# Convert back to original sentiment scale (1 to 5)
predicted_labels += 1

test['label'] = predicted_labels

# Preview the result
print(test.head())


      id                                               text  \
0   1005  If you 're part of her targeted audience , you...   
1  10561  ... Blade II is more enjoyable than the origin...   
2  11048  Spousal abuse is a major problem in contempora...   
3   7846     At best this is a film for the under-7 crowd .   
4  10615  A film that is a portrait of grace in an imper...   

                                        preprocessed  label  
0           you part her targeted audience you cheer      5  
1             blade more enjoyable than the original      5  
2  spousal abuse major problem contemporary socie...      5  
3                 best this film for the under crowd      1  
4           film that portrait grace imperfect world      5  


In [99]:
test.drop(columns=['preprocessed','text']).to_csv('submission.csv', index=False)